In [1]:
%run ../base/base_eq.ipynb
%run ../base/base_plot.ipynb
%run ../base/base_ml.ipynb

In [2]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass

In [3]:
def gen_one_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and the labels are the values of
    the single-peakon solution on a fine grid.
    """
    point, peakon = gen_one_peakon()
    return sol_on_grid(point, peakon)

def gen_two_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a two-peakon solution on a coarse grid, and the labels are the values of
    the two-peakon solution on a fine grid.
    """
    point, twopeakon = gen_two_peakon()
    return sol_on_grid(point, twopeakon)

def gen_one_data():
    return tools.random_function(gen_one_peakon_on_grid, gen_two_peakon_on_grid)

In [8]:
dnns_pros = create_dnns_pros_from_dir('../saved_models/grid_single_peakon', exclude_in=('ScaleDataByBatch',))

INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfcii4v0k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1cf5b950f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfqml29jg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint

In [10]:
X, y = gen_one_peakon_on_grid()
dnns_pros_with_name = tuple((dnn, pro, str(i)) for i, (dnn, pro) in enumerate(dnns_pros))
results = plot_regressors([(Perfect(), None, 'Exact'),
                           *dnns_pros_with_name],
                          X_peak,
                          y_peak)
tuple(results[i].loss for i in range(1, len(dnns_pros)))

FigureCanvasNbAgg()

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpfcii4v0k, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpfqml29jg, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


(24953.583535671627,)